<a href="https://colab.research.google.com/github/SamiraHabib/Segmenting-and-Clustering-Neighborhoods-in-Toronto/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1 #

#  To create the above dataframe: #

* The dataframe will consist of three columns: PostalCode, Borough, and 
Neighborhood

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

* If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe. 

### Submit a link to your Notebook on your Github repository.

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas  to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [22]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

In [23]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text

In [24]:
soup = BeautifulSoup(data,"html5lib")

In [25]:
table = soup.find_all('table')

In [26]:
len(table)

3

# Cleaning the data #

###Removing rows that contain 'Not assigned' and appending the columns to the dataframe

In [37]:
table_contents=[]
table=soup.find('table')
for row in table.find_all('td'):
    col = {}
    if row.span.text=='Not assigned':
        pass
    else:
        col['PostalCode'] = row.p.text[:3]
        col['Borough'] = (row.span.text).split('(')[0]
        col['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(col)


# Create dataframe #

In [42]:
df=pd.DataFrame(table_contents)

In [43]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [44]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [45]:
df.shape

(103, 3)

#Part 2